### Config

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

### Asset

In [ ]:
a = Asset(c.ETH, c.BTC)
print(a.to_dict())
a = Asset.from_symbol('ETH/BTC')
print(a.to_dict())
a = Asset.from_symbol('ETHBTC')
print(a.to_dict())

### Balance

In [ ]:
b = Balance()
print("Currencies",  b.currencies)
print("Entries", b)
b.add_currency(c.ETH)
b.update(c.ETH, delta_free=1.0, delta_used=0.0)
print(b.get(c.ETH))
b.get(c.ETH)[BalanceType.FREE]
Balance.from_dict(b.to_dict())

### Exchange

In [ ]:
# Public informaiton
exchanges = [c.PAPER, c.BINANCE, c.GDAX]#, c.POLONIEX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.timeframes)
    exchange.get_markets()
    exchange.fetch_ohlcv(a, Timeframe.ONE_MIN.id)
    exchange.fetch_order_book(a)
    exchange.fetch_public_trades(a)
    exchange.fetch_ticker(a)

In [ ]:
# Account Information
exchanges = [c.PAPER, c.BINANCE, c.GDAX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    b = exchange.fetch_balance()
    print(exchange.fetch_balance())
    print(exchange.fetch_orders(a))
    print(exchange.fetch_open_orders(a))
    print(exchange.fetch_closed_orders(a))

In [ ]:
exchange = load_exchange(c.PAPER) # c.BINANCE
a = Asset(c.ETH, c.BTC)

# Market BUY
print("Exchange", exchange.id)
order = exchange.create_market_buy_order(a, .01)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
order

In [ ]:
# Check Order
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
found_order = exchange.fetch_order(order.id, a.symbol)
found_order

In [ ]:
# Sell all remaining quantity
curr_balance = exchange.fetch_balance().get(a.base)
print(a.base, curr_balance[BalanceType.TOTAL])
order = exchange.create_market_sell_order(a, curr_balance[BalanceType.TOTAL])
order

In [ ]:
exchange.fetch_balance().get(a.base), exchange.fetch_balance().get(a.quote)

### Dates

In [ ]:
print(Timeframe.FIVE_MIN.delta)
print(Timeframe.FIVE_MIN.id)

### OHLCV Data

In [ ]:
base = c.ETH
quote = c.BTC
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.ETH, c.LTC] ]
period = Timeframe.ONE_MIN
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [ ]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period.id, start, end)
df.head()

In [ ]:
# Load from File
fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period.id)
df = ohlcv.load_chart_data_from_file(fpath)

In [ ]:
# Multiple Coins
ohlcv.download_chart_data(exchange, assets, period.id, start, end)
df = ohlcv.load_multiple_assets(exchange.id, assets, period.id, start)
df.head()

### DataStore

In [ ]:
experiment_name = 'mystrategy1'
fname = 'multiasset'
store = FileStore(os.path.join(cfg.DATA_DIR, experiment_name))

In [ ]:
# DataFrame --> CSV
store.df_to_csv(df, fname)
df = store.csv_to_df(fname, index='time_epoch')
df.head()

In [ ]:
# DataFrame --> JSON
store.df_to_json(df, fname)
df = store.json_to_df(fname, index='time_epoch')
df.head()

In [ ]:
# JSON
dct = {
    'sample': 1,
    'time': datetime.datetime.utcnow()
}
store.save_json(fname, dct)
dct = store.load_json(fname)
dct

### Data Feed

In [ ]:
def get_test_live_feed(exchange_id):
    exchange = load_exchange(exchange_id)
    asset = Asset(c.ETH, c.BTC)
    period = Timeframe.ONE_MIN
    start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
    end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)
    feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period.id)
    feed = ExchangeDataFeed(exchange, [asset], period, start, end=None)
    feed.initialize()
    return feed

def get_test_csv_feed(exchange_id):
    start = datetime.datetime(year=2018, month=1, day=9)
    end = datetime.datetime(year=2018, month=1, day=12)
    asset = Asset(c.ETH, c.BTC)
    period = Timeframe.THIRTY_MIN
    feed_fpath = ohlcv.get_price_data_fpath(asset, exchange_id, period.id)
    feed = CSVDataFeed(feed_fpath, start, end)
    exchange = load_exchange(exchange_id)
    ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period.id, start, end)
    feed.initialize()
    return feed

In [ ]:
# CSV Feed
csv_feed = get_test_csv_feed(c.BINANCE)

# Grab 1 row at a time
for i in range(1):
    data = csv_feed.next()
    print(data['time_utc'], data['close'])    

peek_data = csv_feed.peek()
print(peek_data['time_utc'], peek_data['close'])

# Access all rows in history
csv_feed.history().head()

In [ ]:
# Live Feed
live_feed = get_test_live_feed(c.BINANCE)

# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = live_feed.next(refresh=False)
    print(data['time_utc'], data['close'])

peek_data = live_feed.peek()
print(peek_data['time_utc'], peek_data['close'])

# Refresh data before next query
data = live_feed.next(refresh=True)
print(data['time_utc'], data['close'])

# Access History
live_feed.history(t_minus=3).tail()

### Exchange Data Providers

In [ ]:
csv_feed.next()
dp = FeedExchangeDataProvider(csv_feed)
print(dp.fetch_order_book(asset))
print(dp.fetch_public_trades(asset))
print(dp.fetch_ohlcv(asset, Timeframe.ONE_DAY.id))
print(dp.fetch_ticker(asset))
print(dp.get_markets())

In [ ]:
asset = Asset(c.ETH, c.BTC)
exchange = load_exchange(c.BINANCE)
dp = CCXTExchangeDataProvider(exchange)
print(dp.fetch_order_book(asset)['bids'][:1])
print(dp.fetch_public_trades(asset)[:1])
print(dp.fetch_ohlcv(asset, Timeframe.ONE_DAY.id)[:1])
print(dp.fetch_ticker(asset))
print(dp.get_markets()['ETH/BTC'])

### OrderType

In [ ]:
assert OrderType.from_type_side('limit','buy') == OrderType.LIMIT_BUY
assert OrderType.from_type_side('limit','sell') == OrderType.LIMIT_SELL
assert OrderType.from_type_side('market','buy') == OrderType.MARKET_BUY
assert OrderType.from_type_side('market','sell') == OrderType.MARKET_SELL
o = OrderType.LIMIT_BUY
assert o.type == 'limit' and o.side == 'buy'

In [ ]:
assert OrderType.LIMIT_BUY in OrderType.buy_types()
assert OrderType.MARKET_BUY in OrderType.buy_types()
assert OrderType.LIMIT_SELL in OrderType.sell_types()
assert OrderType.MARKET_SELL in OrderType.sell_types()

In [ ]:
assert OrderType.LIMIT_BUY.is_buy()
assert OrderType.MARKET_BUY.is_buy()
assert OrderType.LIMIT_SELL.is_sell()
assert OrderType.MARKET_SELL.is_sell()
OrderType.LIMIT_BUY.name, OrderType.LIMIT_BUY.value

### Order

In [ ]:
asset = Asset(c.LTC, c.USDT)
order = Order(
    exchange_id=c.PAPER, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
Order.from_dict(order.to_dict())

### Order Manager

In [ ]:
asset = Asset(c.LTC, c.BTC)
exchange = load_exchange(c.PAPER)
exchange.balance = Balance(c.BTC, 5.0)
o1 = Order(exchange.id, asset, price=.01, quantity=1.0, 
           order_type=OrderType.LIMIT_BUY, order_id='1')
o2 = Order(exchange.id, asset, price=.01, quantity=1.0, 
           order_type=OrderType.LIMIT_SELL, order_id='2')
orders = {
    o1.id: o1,
    o2.id: o2
}
orders,exchange.fetch_balance(), exchange.fetch_ticker(asset)['ask']

In [ ]:
order = order_manager.place_order(exchange, orders[o1.id])
orders[o1.id] = order
orders,exchange.fetch_balance()

In [ ]:
resp = order_manager.place_order(exchange, orders[o2.id])
orders[o2.id] = resp
orders,exchange.balance

In [ ]:
limit_buy = order_manager.build_limit_buy_order(exchange, asset, quantity=1.0, price=.1)
limit_sell = order_manager.build_limit_sell_order(exchange, asset, quantity=1.0, price=.1)
market_buy = order_manager.build_market_buy_order(exchange, asset, quantity=1.0)
market_sell = order_manager.build_market_sell_order(exchange, asset, quantity=1.0)
orders = [limit_buy, limit_sell, market_buy, market_sell]
orders

In [ ]:
results = []
for order in orders:
    res = order_manager.place_order(exchange, order)
    results.append(res)
results

In [ ]:
exchange.fetch_balance()

In [ ]:
exchange.fetch_orders(asset)

In [ ]:
exchange.fetch_order('32687cf2c22645fbbd2d639be8a76256', asset)

In [ ]:
updated_orders = exchange.fetch_orders(asset)
ex_order_ids = [order.ex_order_id for order in updated_orders]
print(ex_order_ids)
updated_orders

In [ ]:
ex_orders = order_manager.get_orders(exchange, ex_order_ids, assets=asset)
print(ex_orders)

In [ ]:
print("PENDING", order_manager.get_pending_orders(ex_orders))
print("CANCELED", order_manager.get_canceled_orders(ex_orders))
print("FILLED", order_manager.get_filled_orders(ex_orders))

### Position

In [ ]:
asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1, cost_price=250.0)
print("Asset", pos.asset.to_dict())
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(-1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Go short
pos.update(-2, 150)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Close the short for a LOSS and go long (price went up, we had to pay $300 to cover our short)
# then we bought another share because we're newbs
pos.update(2, 300)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

In [ ]:
"""
Assume that an investor made the following consecutive fund purchases in a taxable account: 1,500 shares at $20, 1,000 shares at $10 and 1,250 shares at $8. The investor’s average cost basis is calculated by dividing $50,000/3,750 shares. The average cost is $13.33.

Suppose the investor then sells 1,000 shares of the fund at $19.

Gain/loss using average cost basis: ($19 - $13.33) x 1,000 shares = $5,667
"""

asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1500, cost_price=20.0)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1000, txn_price=10)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1250, txn_price=8)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

last_cost = pos.cost_price
pos.update(txn_quantity=-1000, txn_price=19)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
print("Profit", (19 - last_cost) * 1000)

In [ ]:
Position.from_dict(pos.to_dict()).to_dict()

### PerformanceTracker

In [ ]:
asset = Asset(c.BTC, c.USD)
perf = PerformanceTracker(starting_cash=5000, timeframe=Timeframe.ONE_MIN)
perf.to_dict()

In [ ]:
# Buy 1 BTC for $1000
pos1 = Position(asset, quantity=1, cost_price=1000.0)
positions = [pos1]
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased $100
Position Return
    Return = .10
    PnL = $100
Cumulative Return
    Return = $100 / $5000 = .02
    PnL = $100
"""
pos1.latest_price = 1100
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased again $400
Position Return
    Return = .5
    PnL = $500
Cumulative Return
    Return = $500 / $5000 = .1
    PnL = $500
"""
pos1.latest_price = 1500
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
# Buy 1 ETH for $500
asset = Asset(c.ETH, c.USD)
pos2 = Position(asset, quantity=1, cost_price=500.0)
positions.append(pos2)
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of ETH decreased $100
Position Return
    Return = -100/500
    PnL = -100
Cumulative Return
    Return = 400 / 5000 = .08
    PnL = 400
"""
pos2.latest_price = 400
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
PerformanceTracker.from_dict(perf.to_dict())

### Portfolio

In [ ]:
starting_cash = 5000
exchange = load_exchange(c.PAPER)
perf = PerformanceTracker(starting_cash, Timeframe.ONE_MIN)
portfolio = Portfolio(starting_cash, perf)

In [ ]:
# Buy 1 BTC for $1000
asset = Asset(c.BTC, c.USD)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1000., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

In [ ]:
portfolio.update(filled_orders=[order])
portfolio

In [ ]:
# Buy more BTC, price has risen $500
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
portfolio.update(filled_orders=[order])
portfolio

In [ ]:
# Sell all BTC at profit
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=2, 
    order_type=OrderType.LIMIT_SELL
)
portfolio.update(filled_orders=[order])
portfolio

In [ ]:
Portfolio.from_dict(portfolio.to_dict())

### Record

In [ ]:
feed = get_test_csv_feed(c.PAPER)

metrics = {
    'SMA': [1, 2, 3, 4],
    'RSI': [3, 4, 2, 1]
}

o1 = Order(c.PAPER, Asset(c.ETH, c.BTC), 5.0, 100.0, OrderType.LIMIT_BUY, order_id='1')
o2 = Order(c.PAPER, Asset(c.LTC, c.BTC), 5.0, 100.0, OrderType.LIMIT_BUY, order_id='2')
orders = {
    o1.id: o1,
    o2.id: o2
}

In [ ]:
root_dir = os.path.join(cfg.DATA_DIR, 'default')
store = FileStore(root_dir)

record = Record(config={}, portfolio=portfolio, balance=Balance(), store=store)
record.orders = orders
record.metrics = metrics
record.ohlcv = feed.history()
record.save()
record = Record.load(root_dir)

In [ ]:
print(record.config)
print(record.metrics)
print(record.balance)
print(record.orders)
print(record.ohlcv.head()[['time_utc','close']])
print(record.portfolio)

### Context

In [ ]:
# From Objects

starting_cash = 5000
exchange = load_exchange(c.PAPER)
store = FileStore(os.path.join(cfg.DATA_DIR, 'default'))
perf = PerformanceTracker(starting_cash, Timeframe.THIRTY_MIN)
portfolio = Portfolio(starting_cash, perf)
feed = get_test_csv_feed(c.PAPER)
config = {'experiment':'default'}
record = Record(config=config, 
                portfolio=portfolio, 
                balance=Balance(), 
                store=store)

In [ ]:
ctx = Context(exchange, feed, record)
print(ctx.record.config)
print(ctx.record.metrics)
print(ctx.record.balance)
print(ctx.record.orders)
print(ctx.record.ohlcv)
print(ctx.record.portfolio)

In [ ]:
# From Config

## TODO - Get this working
trading_config = get_default_backtest_config('default', ['ETH/BTC'])
trading_config

In [ ]:
## TODO - Get this working
ctx = Context.from_config(trading_config)
print(ctx.record.config)
print(ctx.record.metrics)
print(ctx.record.balance)
print(ctx.record.orders)
print(ctx.record.ohlcv)
print(ctx.record.portfolio)

### Strategy

In [ ]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics
strategy = SimpleStrategy()

In [ ]:
config = default_config(TradingMode.BACKTEST)
shutil.rmtree(os.path.join(cfg.DATA_DIR, config['experiment']), ignore_errors=True)
ctx = Context.from_config(config)

In [ ]:
orders = strategy.process(ctx.feed.next(), ctx)

In [ ]:
resp = order_manager.place_orders(ctx.exchange, orders)
resp

In [ ]:
# We're not updating the virtual balance, only the exchange 
# (which is fine until we want to have a multi-exchange algo)
print(ctx.exchange.balance)

### Runner

In [ ]:
mystrategy = SimpleStrategy()
runner.backtest("default", mystrategy)

### ChartDataProviders

In [ ]:
root = os.path.join(cfg.DATA_DIR, 'default')
rp = RecordChartDataProvider(root)

In [ ]:
# TODO
print("OHLCV\n", rp.get_ohlcv())
print("\nPERFORMANCE\n", rp.get_performance())
print("\nRETURNS\n", rp.get_returns())
print("\nPNL\n", rp.get_pnl())

print("\nBALANCE\n", rp.get_balance())
print("\nBALANCE_DICT\n", rp.get_balance_dct())
print("\nPOSITIONS\n", rp.get_positions())
print("\nPOSITIONS_DICT\n", rp.get_positions_dct())

print("\nORDERS\n", rp.get_orders())
print("\nORDERS_DICT\n", rp.get_orders_dct())
print("\nMETRICS\n", rp.get_metrics())

### Charts

In [ ]:
feed = get_test_csv_feed(c.PAPER)

In [ ]:
# We need to increment the feed to start getting history data
for i in range(10):
    _ = feed.next()
start = feed.history().iloc[0]['time_utc']
end = feed.history().iloc[-1]['time_utc']

In [ ]:
# Basic plot
punisher.utils.charts.plot_range(feed.history(), start, end, 'close')

### Dash

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

periods = record.portfolio.perf.periods
df = pd.DataFrame([
    [p['end_time'], p['pnl']] for p in periods
], columns=['time_utc','pnl'])
df

In [ ]:
positions = pd.DataFrame([p.to_dict() for p in record.portfolio.positions])
dct = [p.to_dict() for p in record.portfolio.positions]
{p['asset']: p for p in dct}

In [ ]:
cols = ['coin', 'free', 'used', 'total']
balance = Balance()
coins = balance.currencies
dct = balance.to_dict()
df = pd.DataFrame(
    [[c, dct[c]['free'], dct[c]['used'], dct[c]['total']] for c in coins],
    columns=cols
)

df